In [ ]:
import opendatasets as od
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F



In [ ]:
od.download("https://www.kaggle.com/datasets/milesh1/35-million-chess-games?select=all_with_filtered_anotations_since1998.txt")

Skipping, found downloaded files in "./35-million-chess-games" (use force=True to force download)


In [ ]:
f = open("/content/35-million-chess-games/all_with_filtered_anotations/all_with_filtered_anotations_since1998.txt",'r')
dataset = list(f.readlines())
f.close()


In [ ]:
with torch.no_grad():
    Xtr = []
for k in range(100000):
    game = (dataset[k+5].split("### "))[1]
    temp = []
    for i in range(len(game)):
        if game[i] ==".":
            j=i
            while game[j]!=' ':
                j+=1
            tempstr = game[i+1:j]
            temp.append(tempstr)
    Xtr.append(temp)


In [ ]:
vocab = ['.']
with torch.no_grad():
    Xtr = []
for k in range(len(dataset)-5):
    game = (dataset[k+5].split("### "))[1]

    for i in range(len(game)):
        if game[i] ==".":
            j=i
            while game[j]!=' ':
                j+=1
            tempstr = game[i+1:j]
            if tempstr not in vocab:
                vocab.append(tempstr)

In [ ]:
print(len(vocab))

9675


In [ ]:
def encode(moves:list):
    arr = []
    for i in moves:
        arr.append(vocab.index(i))
    return arr


In [ ]:
X_data = []
Y_data = []

for game in Xtr:
    if len(game)<8:
        continue
    X_data.append(encode(['.','.','.','.','.','.','.',game[0]]))
    X_data.append(encode(['.','.','.','.','.','.',game[0],game[1]]))
    X_data.append(encode(['.','.','.','.','.',game[0],game[1],game[2]]))
    X_data.append(encode(['.','.','.','.',game[0],game[1],game[2],game[3]]))
    X_data.append(encode(['.','.','.',game[0],game[1],game[2],game[3],game[4]]))
    X_data.append(encode(['.','.',game[0],game[1],game[2],game[3],game[4],game[5]]))
    X_data.append(encode(['.',game[0],game[1],game[2],game[3],game[4],game[5],game[6]]))

    Y_data.append(encode([game[1]]))
    Y_data.append(encode([game[2]]))
    Y_data.append(encode([game[3]]))
    Y_data.append(encode([game[4]]))
    Y_data.append(encode([game[5]]))
    Y_data.append(encode([game[6]]))
    Y_data.append(encode([game[7]]))

    for i in range(len(game)-8):
        #print(game[i:i+8],game[i+8])

        X_data.append(encode(game[i:i+8]))
        Y_data.append(encode([game[i+8]]))



In [ ]:
Xtr = 0

In [ ]:
X_data = torch.tensor(X_data)
Y_data = torch.tensor(Y_data)

In [ ]:
# now i have training data and test data
len(X_data)

74354

In [ ]:
print(len(X_data))

In [ ]:
voc_sz = len(vocab)

In [ ]:
class Reshaper(nn.Module):
    def __init__(self,n):
        self.n = n
        super(Reshaper, self).__init__()

    def forward(self, x):
        B, T, C = x.shape
        x = x.view(B, T//self.n, C*self.n)
        if x.shape[1] == 1:
            x = x.squeeze(1)
        self.out = x
        return self.out


In [ ]:
blk_sz = 8
emb_sz = 16
n_fl = 200
voc_sz = len(vocab)
inp_sz = 8
Model = nn.Sequential(
    nn.Embedding(voc_sz,emb_sz),     #  Gets the input ([10000,8,16])
    Reshaper(2),  # becomes [10000,4,32]  # [inp_sz,blk_sz//2,emb_sz*2]
    nn.Linear(emb_sz*2,n_fl),       # takes [32,200]  becomes [10000,4,200]
    nn.Tanh(),
    Reshaper(2),  # becomes [10000,2,400]   # [inp_sz,blk_sz//4,n_fl*2]
    nn.Linear(n_fl*2,n_fl),    # becomes  [10000,2,200]
    nn.Tanh(),
    Reshaper(2),  # becomes [10000,400]  # [inp_sz,n_fl*2]
    nn.Linear(n_fl*2,voc_sz),    # become [400,voc_sz]
    nn.Softmax(),
)

In [ ]:
print(Y_data[2])

tensor([4])


In [ ]:
for x in range(10):
    for x in range(20000):
        ix = torch.randint(0, X_data.shape[0], (8,))
        Xb, Yb = X_data[ix], Y_data[ix] # batch X,Y
        # print(Xb)
        # break
        logits = Model(Xb)
        #print(logits , Yb.squeeze(dim = 1))
        loss = F.cross_entropy(logits, Yb.squeeze(dim = 1))
        Model.zero_grad()
        optimizer = torch.optim.SGD(Model.parameters(), lr=0.01)
        loss.backward()

        optimizer.step()

    print(loss)


tensor(9.1773, grad_fn=<NllLossBackward0>)


KeyboardInterrupt: ignored

In [ ]:
print(shaper)

[torch.Size([8, 4, 32]), torch.Size([8, 2, 400]), torch.Size([8, 1, 400])]


In [ ]:
logi = Model(X_data[5:13])
print(logi,Y_data[5:13])

In [ ]:
# sampeling from the data
# lets start with white

In [ ]:
vocab[1]

'd4'

In [ ]:
context = [0,0,0,0,0,0,0,1]
for x in range(100):
    logits = Model(torch.tensor([context[-8:]]))
    probs = F.softmax(logits, dim=1)
    # sample from the distribution
    ix = torch.multinomial(probs, num_samples=1).item()
    print(vocab[ix])

    move = input("Your Move : ")
    context.append(vocab.index(move))

Nf6
Your Move : Nc3
d6
Your Move : Bf4
d6
Your Move : Bf4
Ne4
Your Move : Nxe4
Nxe4
Your Move : Nxe4
Nxe4
Your Move : Nxe4
Qxe4
Your Move : Qd3
Qxe2
Your Move : Bxe2
Bxc4
Your Move : Qxc4
Nb6
Your Move : Qxc7
Kg7
Your Move : Qxb7
f4
Your Move : Qxa8
Bxf4
Your Move : Qxf8+
Bd6
Your Move : Qxe7+
Rh5
Your Move : Qxh5
Rxa5
Your Move : Qxa5
Bxd4
Your Move : Qxa7+
Ne2
Your Move : Qxg7
Nc6
Your Move : Qxd4
Rh7
Your Move : Qxd6
Rxe8
Your Move : Qxe8
h6
Your Move : Qxc6
b5
Your Move : Ra8#


ValueError: ignored